In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import os
import torchsummary

In [ ]:
data_dir = 'G:/archive'
#csv_file = os.path.join(data_dir, 'birds.csv')
excel_file = os.path.join(data_dir, 'birds.xlsx')
df = pd.read_excel(excel_file)

In [ ]:
df

In [ ]:
train_dir = (data_dir + '/' + df[df['data set'] == 'train']['filepaths']).values
valid_dir = (data_dir + '/' + df[df['data set'] == 'valid']['filepaths']).values
test_dir = (data_dir + '/' + df[df['data set'] == 'test']['filepaths']).values

In [ ]:
# Class id => Label Data
train_data = (df[df['data set'] == 'train']['class id']).values
valid_data =(df[df['data set'] == 'valid']['class id']).values
test_data = (df[df['data set'] == 'test']['class id']).values

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 이미지 정규화
])

class CustomDataset(Dataset):
    def __init__(self, data, data_dir, transform):
        self.data = data    # label data
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image_path = self.data_dir[idx]
        image = Image.open(image_path)
        image = self.transform(image)
        label = self.data[idx]
        
        return image, label

In [ ]:
train_dataset = CustomDataset(train_data, train_dir, transform=transform)
valid_dataset = CustomDataset(valid_data, valid_dir, transform=transform)
test_dataset = CustomDataset(test_data, test_dir, transform=transform)
train_dataset.__getitem__(84532) # (feature, label)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Conv2d(input_channel, output_channel, kernel_size, stride, padding)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 28 * 28, 1024)
        self.fc2 = nn.Linear(1024, 525)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from tqdm import tqdm

for epoch in tqdm(range(10)):
    running_loss = 0.0
    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
               
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f'Epoch {epoch+1} Loss: {running_loss / len(train_loader)}')